# Data Preparation for GeSAI - AB Data Challenge 

In [364]:
# Import necessary libraries
import pandas as pd

# 1. Official Data. Preparación inicial del dataset AB3  

Dataset information data_ab3: Períodes de fuites detectades (tant per l’operadora com pel client), data de
requeriment aixecat, classificació de la incidència i comunicació amb el client (vía i missatge).

In [365]:
# Load dataset from data/official-data/data_ab3.parquet
df_ab3 = pd.read_parquet('../data/official-data/data_ab3.parquet')
print(df_ab3.head())

  POLIZA_SUMINISTRO NUMEROSERIECONTADOR  CONSUMO_REAL          FECHA_HORA  \
0  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           9.0 2024-01-01 00:29:14   
1  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 01:29:14   
2  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB          10.0 2024-01-01 02:29:14   
3  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 03:29:14   
4  U2DVJQEKG3Y56QXB    62TNP5RI2GUII6WB           7.0 2024-01-01 04:29:14   

  DATA_INI_FACT DATA_FIN_FACT CREATED_MENSAJE CODIGO_MENSAJE TIPO_MENSAJE  
0    2024-01-24    2024-03-26             NaT           None         None  
1    2024-01-24    2024-03-26             NaT           None         None  
2    2024-01-24    2024-03-26             NaT           None         None  
3    2024-01-24    2024-03-26             NaT           None         None  
4    2024-01-24    2024-03-26             NaT           None         None  


## 1.1. Información básica de nuestro dataset

In [366]:
# Display basic information about the DataFrame
print('DataFrame Information:')
print(df_ab3.info())

# Print number of null values in each column
print('\nNumber of null values in each column:')
print(df_ab3.isnull().sum())

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121834 entries, 0 to 121833
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   POLIZA_SUMINISTRO    121834 non-null  object        
 1   NUMEROSERIECONTADOR  121834 non-null  object        
 2   CONSUMO_REAL         106983 non-null  float64       
 3   FECHA_HORA           121834 non-null  datetime64[us]
 4   DATA_INI_FACT        121834 non-null  object        
 5   DATA_FIN_FACT        121834 non-null  object        
 6   CREATED_MENSAJE      99400 non-null   datetime64[us]
 7   CODIGO_MENSAJE       99400 non-null   object        
 8   TIPO_MENSAJE         99400 non-null   object        
dtypes: datetime64[us](2), float64(1), object(6)
memory usage: 8.4+ MB
None

Number of null values in each column:
POLIZA_SUMINISTRO          0
NUMEROSERIECONTADOR        0
CONSUMO_REAL           14851
FECHA_HORA                 

## 1.2. Tratamiento de los valores nulos

### 1.2.1. Imputación en valores nulos de la variable "CONSUMO_REAL"
Dada la información proporcionada por el equipo de AB Data, cuando la variable "CONSUMO_REAL" tiene un valor NaN quiere decir que el valor registrado es un valor < 1, para el correcto funcionamiento de los modelos a desarrollar asumiremos que este valor NaN será "0".

In [367]:
# Imputation of missing values in "CONSUMO_REAL" column
df_ab3.fillna({'CONSUMO_REAL': 0}, inplace=True)

### 1.2.2. Toma de decisiones sobre las variables "CREATED_MENSAJE", "CODIGO_MENSAJE" y "TIPO_MENSAJE"

Estas 3 variables hacen referencia a la detección y comunicación de fuga ('FUITA'), así como a su reiteración ('REITERACIÓ DE FUITA). Para llevar a cabo nuestro modelo predictivo nos hará falta simplificar el proceso de detección de fuga por lo que se ha decidido retirar estas columnas y añadir una nueva columna binaria que indica 0 si no hay fuga y 1 si hay fuga.

In [368]:
# Create binary column for leak detection
df_ab3['FUGA_DETECTADA'] = df_ab3['CODIGO_MENSAJE'].apply(lambda x: 1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)

# Drop unnecessary columns
df_ab3.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'], inplace=True)

### 1.2.3. Verificación de la imputación en valores nulos

Verificamos si hemos realizado una correcta imputación imprimiendo el número de valores nulos en cada columna.

In [369]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_ab3.isnull().sum())

Number of null values in each column:
POLIZA_SUMINISTRO      0
NUMEROSERIECONTADOR    0
CONSUMO_REAL           0
FECHA_HORA             0
DATA_INI_FACT          0
DATA_FIN_FACT          0
FUGA_DETECTADA         0
dtype: int64


## 1.3. Tratamiento de la variable "FECHA_HORA" 

Para una mayor claridad de nuestros datos se ha decidido separar la variable "FECHA_HORA" en dos variables distintas: "FECHA" y "HORA".


In [370]:
# Convert specified columns to datetime format
for col in ["FECHA_HORA"]:
    df_ab3[col] = pd.to_datetime(df_ab3[col], errors="coerce")

# Split 'FECHA_HORA' into separate date and time columns
df_ab3['FECHA'] = df_ab3['FECHA_HORA'].dt.date
df_ab3['HORA'] = df_ab3['FECHA_HORA'].dt.time

# Cast 'FECHA' column to datetime
df_ab3['FECHA'] = pd.to_datetime(df_ab3['FECHA'], errors="coerce")

# Cast 'HORA' column to datetime
df_ab3['HORA'] = pd.to_datetime(df_ab3['HORA'].astype(str), format='%H:%M:%S', errors='coerce').dt.time

# Drop the original 'FECHA_HORA' column
df_ab3 = df_ab3.drop(columns=['FECHA_HORA'])

## 1.4. Ordenar las columnas del dataset

In [371]:
# Order columns
column_order = ['POLIZA_SUMINISTRO', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'CONSUMO_REAL', 'FUGA_DETECTADA']
df_ab3 = df_ab3[column_order]

# Display updated DataFrame 
print('Updated DataFrame:')
display(df_ab3.head())

Updated DataFrame:


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,FECHA,HORA,CONSUMO_REAL,FUGA_DETECTADA
0,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,00:29:14,9.0,0
1,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,01:29:14,7.0,0
2,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,02:29:14,10.0,0
3,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,03:29:14,7.0,0
4,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,04:29:14,7.0,0


# 2. Open Data

## 2.1. Datos metereológicos

In [372]:
# Load meteorological data
df_aemet1 = pd.read_json('../data/open-data/data_aemet_1.json')

df_aemet2 = pd.read_json('../data/open-data/data_aemet_2.json')


# Merge meteorological datasets
df_aemet = pd.concat([df_aemet1, df_aemet2], ignore_index=True)

print('Meteorological DataFrame:')
display(df_aemet)

Meteorological DataFrame:


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,6","0,0","8,6",06:00,"14,6",13:40,36.0,"1,9","4,2",06:20,82,90,16:50,69,12:30
1,2024-01-02,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,8","0,0","8,1",01:50,"15,4",12:20,26.0,"2,8","7,5",11:20,53,81,00:30,39,16:10
2,2024-01-03,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"16,0","0,0","13,6",00:30,"18,4",12:30,27.0,"1,4","9,7",11:50,74,86,19:50,62,00:00
3,2024-01-04,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"14,2","0,3","11,0",07:00,"17,4",13:30,26.0,"1,4","4,7",19:00,89,94,18:10,79,00:00
4,2024-01-05,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,2","8,4","8,9",21:50,"13,5",13:50,30.0,"2,2","6,9",23:30,89,94,Varias,78,13:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2024-12-27,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"12,5","0,0","7,9",23:59,"17,1",12:20,2.0,"1,7","6,4",04:50,62,95,Varias,39,14:20
362,2024-12-28,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"9,7","0,0","4,6",07:30,"14,8",13:30,20.0,"1,9","5,6",14:30,73,88,20:50,59,12:40
363,2024-12-29,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"10,6","0,0","5,1",07:40,"16,2",13:00,99.0,"2,5","5,0",Varias,69,86,03:40,46,12:40
364,2024-12-30,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"9,3","0,0","4,4",07:40,"14,2",13:00,21.0,"2,2","5,6",14:00,76,89,20:50,59,11:40


### 2.1.1. Comprobación de valores nulos del dataset

In [373]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_aemet.isnull().sum())

Number of null values in each column:
fecha         0
indicativo    0
nombre        0
provincia     0
altitud       0
tmed          0
prec          1
tmin          0
horatmin      1
tmax          0
horatmax      0
dir           1
velmedia      0
racha         1
horaracha     1
hrMedia       0
hrMax         0
horaHrMax     0
hrMin         0
horaHrMin     0
dtype: int64


Observamos que es un buen dataset por el poco número de valores nulos así que podemos utilizarlo de forma efectiva para nuestro dataset.


### 2.1.2. Selección de características relevantes para nuestro dataset principal

Analizaremos todas las variables medidas en cada estación de medición de nuestro dataset para seleccionar las más relevantes para nuestro objetivo.

**Variables Escogidas**
Nos centramos en filtrar el "ruido" (falsos positivos) causado por el clima cálido de Barcelona y en capturar causas directas de rotura.

- fecha: IMPRESCINDIBLE. Es la clave para unir (merge) estos datos climáticos con nuestro dataset de consumo (AB3).

- tmin: MUY IMPORTANTE. Es la temperatura mínima diaria. La usaremos para calcular TEMP_MIN_BCN para detectar heladas.

- tmax: MUY IMPORTANTE. Es la temperatura máxima diaria. La usaremos para calcular TEMP_MAX_BCN para filtrar el "ruido" por calor.

- prec: IMPORTANTE. Es la precipitación acumulada diaria. La usaremos para calcular PRECIP_MEAN_BCN como el "anulador del ruido" por calor.

- hrMedia: ÚTIL. Es la humedad relativa media diaria. La podemos usar (calculando la mean() entre estaciones) para afinar el contexto del calor (calor seco vs. calor húmedo).

In [374]:
# Select relevant features for our main dataset
relevant_features = ['fecha', 'tmed', 'tmin', 'tmax', 'prec', 'hrMedia']
df_aemet = df_aemet[relevant_features]
print('Selected relevant features for meteorological dataset:')
display(df_aemet.head())

Selected relevant features for meteorological dataset:


,fecha,tmed,tmin,tmax,prec,hrMedia
0,2024-01-01,"11,6","8,6","14,6","0,0",82
1,2024-01-02,"11,8","8,1","15,4","0,0",53
2,2024-01-03,"16,0","13,6","18,4","0,0",74
3,2024-01-04,"14,2","11,0","17,4","0,3",89
4,2024-01-05,"11,2","8,9","13,5","8,4",89


### 2.1.3. Imputación en valores nulos en la variable "prec"

In [375]:
print('Number of null values in each column after feature selection:')
print(df_aemet.isnull().sum())

# Imputation in null value of prec column with 0
df_aemet.fillna({'prec': 0}, inplace=True)

print('\nNumber of null values in each column after imputation:')
print(df_aemet.isnull().sum())

# Print number of null values
print('\nRows with null values:')
print(df_aemet.isnull().sum())


Number of null values in each column after feature selection:
fecha      0
tmed       0
tmin       0
tmax       0
prec       1
hrMedia    0
dtype: int64

Number of null values in each column after imputation:
fecha      0
tmed       0
tmin       0
tmax       0
prec       0
hrMedia    0
dtype: int64

Rows with null values:
fecha      0
tmed       0
tmin       0
tmax       0
prec       0
hrMedia    0
dtype: int64


### 2.1.4. Merge con el dataset principal

In [376]:
# Rename columns for clarity
cols_to_rename = {
    'fecha': 'FECHA',
    'tmed': 'TEMP_MEDIA',
    'tmin': 'TEMP_MIN',
    'tmax': 'TEMP_MAX',
    'prec': 'PRECIPITACION',
    'hrMedia': 'HUMEDAD_RELATIVA_MEDIA'
}
df_aemet.rename(columns=cols_to_rename, inplace=True)

# Cast FECHA columns to datetime
df_aemet['FECHA'] = pd.to_datetime(df_aemet['FECHA'], errors='coerce')

# Merge with main dataset
df_final = pd.merge(df_ab3, df_aemet, on='FECHA', how='left')

In [377]:
# Order columns
column_order_final = ['POLIZA_SUMINISTRO', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA','CONSUMO_REAL', 'FUGA_DETECTADA']
df_final = df_final[column_order_final]


print('Final merged DataFrame:')
display(df_final.head())

Final merged DataFrame:


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,FECHA,HORA,TEMP_MEDIA,TEMP_MIN,TEMP_MAX,PRECIPITACION,HUMEDAD_RELATIVA_MEDIA,CONSUMO_REAL,FUGA_DETECTADA
0,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,00:29:14,"11,6","8,6","14,6","0,0",82,9.0,0
1,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,01:29:14,"11,6","8,6","14,6","0,0",82,7.0,0
2,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,02:29:14,"11,6","8,6","14,6","0,0",82,10.0,0
3,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,03:29:14,"11,6","8,6","14,6","0,0",82,7.0,0
4,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,04:29:14,"11,6","8,6","14,6","0,0",82,7.0,0


## 2.2. Gestión de días festivos

A continuación crearemos una variable booleana a través de la librería holidays para almacenar los días festivos puesto que este hecho se relacionará directamente con el consumo de agua.

In [378]:
#%pip install holidays

In [379]:
# pip install holidays
import holidays

### 2.2.1. Selección de días festivos y findes de semana

In [380]:
# Create a range of dates
fechas = pd.date_range(start='2024-01-01', end='2024-12-31')
df_fechas = pd.DataFrame({'FECHA': fechas})

# Get holidays for Spain, subdivision Catalonia (ES, CT)
es_holidays = holidays.CountryHoliday('ES', subdiv='CT', years=2024)

# Create the 'FESTIVO' column
df_fechas['FESTIVO'] = df_fechas['FECHA'].apply(lambda date: date in es_holidays)

### 2.2.2. Tener en cuenta días no laborales cotidianos

In [381]:
def get_tipo_dia_simple(fecha, es_festivo):
    if es_festivo: 
        return 'Festivo'
    elif fecha.weekday() >= 5: 
        return 'Fin de Semana'
    else:
        return 'Laborable'

df_fechas['TIPO_DIA'] = df_fechas.apply(lambda row: get_tipo_dia_simple(row['FECHA'], row['FESTIVO']), axis=1)

### 2.2.2. Merge con el dataset principal

In [382]:
# Cast FECHA column to datetime
df_fechas['FECHA'] = pd.to_datetime(df_fechas['FECHA'], errors='coerce')

# Merge holiday information with final dataset
df_final = pd.merge(df_final, df_fechas[['FECHA', 'FESTIVO', 'TIPO_DIA']], on='FECHA', how='right')

# Order columns 
column_order_final = ['POLIZA_SUMINISTRO', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'FESTIVO', 'TIPO_DIA', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA','CONSUMO_REAL', 'FUGA_DETECTADA']
df_final = df_final[column_order_final] 

df_final.head()


,POLIZA_SUMINISTRO,NUMEROSERIECONTADOR,FECHA,HORA,FESTIVO,TIPO_DIA,TEMP_MEDIA,TEMP_MIN,TEMP_MAX,PRECIPITACION,HUMEDAD_RELATIVA_MEDIA,CONSUMO_REAL,FUGA_DETECTADA
0,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,00:29:14,True,Festivo,"11,6","8,6","14,6","0,0",82,9.0,0
1,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,01:29:14,True,Festivo,"11,6","8,6","14,6","0,0",82,7.0,0
2,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,02:29:14,True,Festivo,"11,6","8,6","14,6","0,0",82,10.0,0
3,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,03:29:14,True,Festivo,"11,6","8,6","14,6","0,0",82,7.0,0
4,U2DVJQEKG3Y56QXB,62TNP5RI2GUII6WB,2024-01-01,04:29:14,True,Festivo,"11,6","8,6","14,6","0,0",82,7.0,0


# 3. Download final dataset

In [383]:
# Download final dataset
df_final.to_csv('../data/final_dataset.csv', index=False)
